Mandatory packages to be imported

In [ ]:
!pip install numpy
!pip install yfinance
!pip install pandas

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf

Take in VIX data (1990 till date)

In [10]:
vix_df = yf.download('^VIX')

[*********************100%***********************]  1 of 1 completed


In [12]:
vix_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,17.240000,0
1990-01-03,18.190001,18.190001,18.190001,18.190001,18.190001,0
1990-01-04,19.219999,19.219999,19.219999,19.219999,19.219999,0
1990-01-05,20.110001,20.110001,20.110001,20.110001,20.110001,0
1990-01-08,20.260000,20.260000,20.260000,20.260000,20.260000,0
...,...,...,...,...,...,...
2022-05-23,28.980000,30.389999,28.290001,28.480000,28.480000,0
2022-05-24,29.430000,31.070000,29.040001,29.450001,29.450001,0
2022-05-25,29.330000,30.230000,28.160000,28.370001,28.370001,0


Strategy specification (1-month Moving Average)

In [13]:
vix_df['MA'] = vix_df.Close.rolling('30D').mean()

In [14]:
vix_df

,Open,High,Low,Close,Adj Close,Volume,MA
Date,,,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,17.240000,0,17.240000
1990-01-03,18.190001,18.190001,18.190001,18.190001,18.190001,0,17.715000
1990-01-04,19.219999,19.219999,19.219999,19.219999,19.219999,0,18.216667
1990-01-05,20.110001,20.110001,20.110001,20.110001,20.110001,0,18.690000
1990-01-08,20.260000,20.260000,20.260000,20.260000,20.260000,0,19.004000
...,...,...,...,...,...,...,...
2022-05-23,28.980000,30.389999,28.290001,28.480000,28.480000,0,30.317143
2022-05-24,29.430000,31.070000,29.040001,29.450001,29.450001,0,30.277728
2022-05-25,29.330000,30.230000,28.160000,28.370001,28.370001,0,30.339091


Comparing 'Close' and MA

In [15]:
vix_df_filtered = vix_df[vix_df.Close > 1.5 * vix_df.MA]

In [17]:
vix_df_filtered

,Open,High,Low,Close,Adj Close,Volume,MA
Date,,,,,,,
1990-08-06,35.910000,35.910000,35.910000,35.910000,35.910000,0,20.330476
1990-08-07,32.750000,32.750000,32.750000,32.750000,32.750000,0,20.895000
1994-04-04,23.610001,28.299999,23.610001,23.870001,23.870001,0,15.487000
1997-10-30,36.709999,38.560001,35.840000,38.200001,38.200001,0,23.360909
2001-09-17,43.200001,44.330002,39.770000,41.759998,41.759998,0,25.810625
...,...,...,...,...,...,...,...
2020-03-19,80.620003,84.260002,68.570000,72.000000,72.000000,0,44.842273
2021-01-27,23.820000,37.209999,23.709999,37.209999,37.209999,0,23.929500
2021-11-26,26.620001,28.990000,23.879999,28.620001,28.620001,0,17.673810


Further filter to pick values for timedelta > 30 days

In [22]:
pd.Series(vix_df_filtered.index)

0    1990-08-06
1    1990-08-07
2    1994-04-04
3    1997-10-30
4    2001-09-17
        ...    
60   2020-03-19
61   2021-01-27
62   2021-11-26
63   2021-12-01
64   2022-01-25
Name: Date, Length: 65, dtype: datetime64[ns]

In [23]:
pd.Series(vix_df_filtered.index).diff()

0          NaT
1       1 days
2    1336 days
3    1305 days
4    1418 days
        ...   
60      1 days
61    314 days
62    303 days
63      5 days
64     55 days
Name: Date, Length: 65, dtype: timedelta64[ns]

Convert to float

In [24]:
pd.Series(vix_df_filtered.index).diff() / np.timedelta64(1, 'D')

0        NaN
1        1.0
2     1336.0
3     1305.0
4     1418.0
       ...  
60       1.0
61     314.0
62     303.0
63       5.0
64      55.0
Name: Date, Length: 65, dtype: float64

Filter for 1 month

In [25]:
pd.Series(vix_df_filtered.index).diff() / np.timedelta64(1, 'D') >= 30

0     False
1     False
2      True
3      True
4      True
      ...  
60    False
61     True
62     True
63    False
64     True
Name: Date, Length: 65, dtype: bool

In [26]:
series = pd.Series(vix_df_filtered.index).diff() / np.timedelta64(1, 'D') >= 30

To get around initial value NaT

In [28]:
series[0] = True

In [29]:
series

0      True
1     False
2      True
3      True
4      True
      ...  
60    False
61     True
62     True
63    False
64     True
Name: Date, Length: 65, dtype: bool

Get buying signals

In [32]:
signals = vix_df_filtered[series.values]
signals

,Open,High,Low,Close,Adj Close,Volume,MA
Date,,,,,,,
1990-08-06,35.910000,35.910000,35.910000,35.910000,35.910000,0,20.330476
1994-04-04,23.610001,28.299999,23.610001,23.870001,23.870001,0,15.487000
1997-10-30,36.709999,38.560001,35.840000,38.200001,38.200001,0,23.360909
2001-09-17,43.200001,44.330002,39.770000,41.759998,41.759998,0,25.810625
2007-02-27,12.120000,19.010000,12.100000,18.309999,18.309999,0,10.921905
2008-09-17,31.959999,36.400002,30.250000,36.220001,36.220001,0,23.478095
2010-05-06,25.879999,40.709999,24.430000,32.799999,32.799999,0,18.951364
2011-08-04,24.570000,32.070000,24.309999,31.660000,31.660000,0,20.732273
2014-10-13,21.160000,24.639999,20.520000,24.639999,24.639999,0,15.625714


This specific signal has been triggered 21 times since 1990

In [34]:
signals.shape

(21, 7)

In [37]:
sp500_df = yf.download('^GSPC', start = '1990-01-01')
sp500_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,353.399994,359.690002,351.980011,359.690002,359.690002,162070000
1990-01-03,359.690002,360.589996,357.890015,358.760010,358.760010,192330000
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000
1990-01-08,352.200012,354.239990,350.540009,353.790009,353.790009,140110000
